##### Creation Date:
##### February 19 2022
##### Created By Alperen KOLAMUC

In [1]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics 

In [2]:
import re
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [3]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_feather("tmp/bulldozers_1")
df

,index,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,...,DateWeek,DateDay,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateIs_quarter_end,DateIs_quarter_start,DateIs_year_end,DateIs_year_start
0,205615,1646770,9.159047,1126363,8434,132,18.0,1974,NaN,0,...,3,17,1,17,False,False,False,False,False,False
1,168619,1595295,9.680344,1291243,8298,132,18.0,1977,NaN,0,...,5,31,1,31,True,False,False,False,False,False
2,28927,1259973,8.948976,1150485,6788,132,18.0,1978,NaN,0,...,5,31,1,31,True,False,False,False,False,False
3,63166,1329619,10.221941,436674,4089,132,99.0,1987,NaN,0,...,5,31,1,31,True,False,False,False,False,False
4,140922,1504404,10.165852,1310407,4139,132,99.0,1975,NaN,0,...,5,31,1,31,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339995,23972,1222821,11.512925,1016175,16725,121,3.0,2005,9341.0,1,...,52,29,3,363,False,False,False,False,False,False
339996,23971,1222820,11.512925,1010621,16725,121,3.0,2005,9946.0,1,...,52,29,3,363,False,False,False,False,False,False
339997,23970,1222817,9.305651,999753,25175,121,3.0,2002,7140.0,1,...,52,29,3,363,False,False,False,False,False,False
339998,23968,1222813,11.532728,1038562,13316,121,3.0,2006,1764.0,3,...,52,29,3,363,False,False,False,False,False,False


In [5]:
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum(): # if pd.isnull(col).sum != 0, col.isnull().sum() != 0
            df[name+"_na"] = pd.isnull(col)
        df[name] = col.fillna(col.median())

In [6]:
def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes + 1 # ortak ağız için

In [7]:
def proc_df(df, y_fld):
    y = df[y_fld].values
    df.drop([y_fld], axis=1, inplace=True)

    for n, c in df.items():
        fix_missing(df, c, n)
        numericalize(df, c, n)

    return df, y

In [8]:
proc_df(df, "SalePrice")

(         index  SalesID  MachineID  ModelID  datasource  auctioneerID  \
 0       205615  1646770    1126363     8434         132          18.0   
 1       168619  1595295    1291243     8298         132          18.0   
 2        28927  1259973    1150485     6788         132          18.0   
 3        63166  1329619     436674     4089         132          99.0   
 4       140922  1504404    1310407     4139         132          99.0   
 ...        ...      ...        ...      ...         ...           ...   
 339995   23972  1222821    1016175    16725         121           3.0   
 339996   23971  1222820    1010621    16725         121           3.0   
 339997   23970  1222817     999753    25175         121           3.0   
 339998   23968  1222813    1038562    13316         121           3.0   
 339999   23967  1222811    1049475    13284         121           3.0   
 
         YearMade  MachineHoursCurrentMeter  UsageBand  fiModelDesc  ...  \
 0           1974                 

* Yukarıda ki fonksiyonlar doğru çalışsalarda tam olarak ihtiyacımızı karşılayamıyorlar.
* Sadece ismini verdiğimiz sütuna `fix_missing` ve `numericalize` yapıyorlar.
* Verinin train mi valid mi ayrımını yapamıyorlar
* `Train`'de boş olan yere değer yazıldığında o sünunun adında bir tanede `_NA` sütunu oluyordu model ona göre öğreniyordu. Sonra `DataFrame`'in diğer verileri ile `Validasyon` yaparken belki o değerlerde hiç `NaN` olamayacak. Fakat `Train`'de `NA` diye bir sutun vardı ama `Validasyonda` yok. Bu sefer model sıkıntıya düşecek. Bunun önlemek için `Train` data'sında hangi değerleri eklediysem onları belirten bir **dictionary** oluşturup o **dictionary** ile `Validasyon` işlemimi beslerim. Bu sefer model `Validasyon`'da hangi datalarda **NaN** varmış bilir.

##### Bunun için; 
1. Train'de hangi sütunlar için `_NA` ekledim bulmam lazım
2. Valid ve Test'te boş veri varsa Train'de doldurulduğu gibi doldurulur.
    * eg. Yani Trainde boş verileri doldurmak için ortalama aldık. `SalePrice` diyelim `18` olsun. **Valid**'de de boş kısım için ortalama değer yazacağız ama burada ince bir nüans var. **Valid** de yazacağımız ortalama değer valid deki sutunun ortalaması **değil** yine Trainde ki sütunun ortalama değerini yazmamız gerekiyor.

In [18]:
def fix_missing(df, col, name, nan_dict, is_train):
    if is_train:
        if is_numeric_dtype(col):
            if pd.isnull(col).sum():
                df[name+"_NA"] = pd.isnull(col)
                nan_dict[name] = col.median()
                df[name] = col.fillna(nan_dict[name])

    else:
        if is_numeric_dtype(col):
            if name in nan_dict:
                df[name+"_NA"] = pd.isnull(col)
                df[name] = col.fillna(nan_dict[name])
            
            else:
                df[name] = col.fillna(df[name].median())

In [19]:
def proc_df(df, y_fld, nan_dict=None, is_train=True):
    df = df.copy()
    y = df[y_fld].values

    df.drop([y_fld], axis=1, inplace=True)

    if nan_dict is None:
        nan_dict = {}
    
    for n, c in df.items():
        fix_missing(df, c, n, nan_dict, is_train)
        numericalize(df, c, n)

    if is_train:
        return df, y, nan_dict
    
    return df, y

In [20]:
def split_train_val(df, n):
    return(df[:n].copy(), df[n:].copy())

In [21]:
n_valid = 12000
n_train = len(df)-n_valid
raw_train, raw_valid = split_train_val(df, n_train)

In [32]:
x_train, y_train, nas = proc_df(raw_train, 'SalePrice')

In [33]:
x_valid, y_valid = proc_df(raw_valid, 'SalePrice', nan_dict=nas, is_train=False)

In [30]:
for e in x_train.columns:
    print(e)

index
SalesID
MachineID
ModelID
datasource
auctioneerID
YearMade
MachineHoursCurrentMeter
UsageBand
fiModelDesc
fiBaseModel
fiSecondaryDesc
fiModelSeries
fiModelDescriptor
ProductSize
fiProductClassDesc
state
ProductGroup
ProductGroupDesc
Drive_System
Enclosure
Forks
Pad_Type
Ride_Control
Stick
Transmission
Turbocharged
Blade_Extension
Blade_Width
Enclosure_Type
Engine_Horsepower
Hydraulics
Pushblock
Ripper
Scarifier
Tip_Control
Tire_Size
Coupler
Coupler_System
Grouser_Tracks
Hydraulics_Flow
Track_Type
Undercarriage_Pad_Width
Stick_Length
Thumb
Pattern_Changer
Grouser_Type
Backhoe_Mounting
Blade_Type
Travel_Controls
Differential_Type
Steering_Controls
DateYear
DateMonth
DateWeek
DateDay
DateDayofweek
DateDayofyear
DateIs_month_end
DateIs_month_start
DateIs_quarter_end
DateIs_quarter_start
DateIs_year_end
DateIs_year_start
auctioneerID_NA
MachineHoursCurrentMeter_NA


In [31]:
x_train.MachineHoursCurrentMeter_NA

0          True
1          True
2          True
3          True
4          True
          ...  
327995    False
327996    False
327997    False
327998    False
327999     True
Name: MachineHoursCurrentMeter_NA, Length: 328000, dtype: bool